<a href="https://colab.research.google.com/github/fazzyA/LangBot/blob/main/LinkBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  import openai
  from langchain_openai import ChatOpenAI
except:
    !pip install openai
    !pip install langchain
    !pip install langchain-openai
!pip install os
!pip install openai
!pip install sys
import os
import openai
import sys
sys.path.append('../..')

import panel as pn  # GUI
pn.extension()

import openai
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# llm = ChatOpenAI(openai_api_key=os.getenv('OPENAI_API_KEY'))
openai.api_key  = os.getenv('OPENAI_API_KEY')
# openai.api_key  = os.environ['OPENAI_API_KEY']
llm_name = "gpt-4o-mini"

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
ERROR: Could not find a version that satisfies the requirement sys (from versions: none)
ERROR: No matching distribution found for sys


<ipython-input-95-01136a421a21>:17: UserWarning: Using Panel interactively in Colab notebooks requires the jupyter_bokeh package to be installed. Install it with:

    !pip install jupyter_bokeh

and try again.
  pn.extension()


In [ ]:
!pip install chromadb
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [ ]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

0

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)
llm.predict("Hello world!")

'Hello! How can I assist you today?'

In [ ]:
# Build prompt
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

# Run chain
from langchain.chains import RetrievalQA
question = "Is probability a class topic?"
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

'Yes, probability is often a topic covered in mathematics classes, particularly in statistics and algebra courses. It is fundamental for understanding data analysis and decision-making processes. Thanks for asking!'

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [ ]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [ ]:
question = "Is probability a class topic?"
result = qa({"question": question})

In [ ]:
result['answer']

'Yes, probability is often a class topic in various educational settings, including mathematics and statistics courses. It is a fundamental concept that deals with the likelihood of different outcomes and is commonly taught at different levels, from elementary school to advanced university courses.'

In [ ]:
question = "why are those prerequesites needed?"
result = qa({"question": question})

In [ ]:
result['answer']

"I don't know."

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-community
!pip install langchain.vectorstores

ERROR: Could not find a version that satisfies the requirement langchain.vectorstores (from versions: none)
ERROR: No matching distribution found for langchain.vectorstores


In [ ]:
try:
  from langchain.embeddings.openai import OpenAIEmbeddings
  from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
  from langchain.vectorstores import DocArrayInMemorySearch
  from langchain.document_loaders import TextLoader
  from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
  from langchain.memory import ConversationBufferMemory
  from langchain.chat_models import ChatOpenAI
  from langchain.document_loaders import TextLoader
  from langchain.document_loaders import PyPDFLoader
  import openai
except:
  !pip install langchain
  !pip install langchain-openai
  # !pip install langchain-community
  !pip install langchain.embeddings.openai
  !pip install openai


!pip install panel
!pip install params
!pip install pypdf
!pip install docarray
!pip install openai


In [ ]:
import openai
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# llm = ChatOpenAI(openai_api_key=os.getenv('OPENAI_API_KEY'))
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
def load_db(file, chain_type, k):
    # load documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    print(documents)
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    # define embedding
    embeddings = OpenAIEmbeddings()
    # create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    # define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0),
        chain_type=chain_type,
        retriever=retriever,
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa

In [ ]:
!pip install panel
import panel as pn
import param

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])

    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = "drive/MyDrive/Manual_Link4_Pearl.pdf"
        self.qa = load_db(self.loaded_file,"stuff", 4)
        print(self.loaded_file)
        # load_db("drive/MyDrive/Manual_Link4_Pearl.pdf","stuff", 4)

    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = load_db("temp.pdf", "stuff", 4)
            button_load.button_style="solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    def convchain(self, query):
        # if not query:
        #     return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer']
        # self.panels.extend([
        #     pn.Row('User:', pn.pane.Markdown(query, width=600)),
        #     pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, style={'background-color': '#F6F6F6'}))
        # ])
        # inp.value = ''  #clears loading indicator when cleared
        # return pn.WidgetBox(*self.panels,scroll=True)
        return result['answer']

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history')
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return

In [ ]:
cb = cbfs()

# file_input = pn.widgets.FileInput(accept='.pdf')
# button_load = pn.widgets.Button(name="Load DB", button_type='primary')
# button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
# button_clearhistory.on_click(cb.clr_history)
# inp = pn.widgets.TextInput( placeholder='Enter text here…')

# bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
# conversation = pn.bind(cb.convchain, inp)

# jpg_pane = pn.pane.Image( './img/convchain.jpg')



[Document(metadata={'source': 'drive/MyDrive/Manual_Link4_Pearl.pdf', 'page': 0}, page_content='1.16.930.0\nPearl\nUser\nManual\nVersion\n1.16.930.0\nPage\n1\nof\n139\nLink4\n(714)\n524-0004\n'), Document(metadata={'source': 'drive/MyDrive/Manual_Link4_Pearl.pdf', 'page': 1}, page_content='1.16.930.0\nTable\nof\nContents\n1.\nIntroduction\n7\n1.1\nCustomer\nService\n7\n1.2\nContact\nUs\n7\n2.Terms\nand\nConditions\n8\n2.1\nWarranty\n8\n2.2\nReturns\n8\n2.3\nRepairs\n8\n2.4\nAdditional\nCosts\n8\n2.5\nGoverning\nLaw\n8\n2.6\nSupport\n9\n3.\nBefore\nyou\nBegin\n9\n3.1\nTemperature\nControl\n9\n3.2\nDeadbands\n9\n3.3\nSetpoint\n9\n3.4\nRamping\n10\n3.5\nHumidity\nControl\n11\n3.6\nControl\nStrategy\n11\n3.7\nSetpoint\nand\nStaging\nWorksheets\n12\n3.8\nCO2\nWorksheet\n13\n3.9\nAccumulated\nLight\nWorksheet\nfor\nIrrigation\n14\n3.10\nCycle\n&\nTrigger\nWorksheet\nfor\nIrrigation\n14\n3.11\nSchedule\nWorksheet\nfor\nIrrigation\n15\n4.\nInstallation\n16\n4.1\nContent\nInspection\n16\n4.2\nR

In [ ]:
# cb.call_load_db()
result = cb.convchain("What is pearl controller")
# question = "can you read my file?"
# result = qa({"question": question})

In [ ]:
result

'The Pearl TM system is Link4’s Intelligent Greenhouse Environmental Controller. It represents the latest in greenhouse environmental control automation, enabling users to control and integrate a variety of equipment in their greenhouse. The system is flexible and easy to use, linking together various irrigation needs including heating, cooling, venting, shading, humidity, CO2, lighting, and misting. The Pearl TM is designed specifically for the demands and cost concerns of small to midsize growers, offering a better feature-to-price ratio compared to other controllers. It is a modular system that allows for quick installation and dynamic programming flexibility, improving the quality and efficiency of greenhouse operations.'

In [ ]:
tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
    # pn.Tabs(('Conversation', tab1))

)

In [ ]:
dashboard


Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=Markdown, defer_load=False, height=300, loading_indicator=True)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=Column, defer_load=False)
            [1] Divider()
            [2] ParamMethod(method, _pane=WidgetBox, defer_load=False)
        [2] Column
            [0] ParamMethod(method, _pane=WidgetBox, defer_load=False)
            [1] Divider()
        [3] Column
            [0] Row
                [0] FileInput(accept='.pdf')
                [1] Button(button_type='primary', name='Load DB')
                [2] ParamFunction(function, _pane=Markdown, defer_load=False)
            [1] Row
                [0] Button(button_type='warning', name='Clear History')
                [1] Markdown(str)
            [2] Divider()
            [3] Row
                [0] Image(str, width=400)